### Week 9: Creating macro variables using PROC SQL


#### Create a macro variable using PROC SQL INTO and then pass it into a DATA step 
##### The PROC SQL code below uses the SELECT and INTO statements to store the value of average weight [ave(weight)] in a macro variable. 

In [3]:
*Ex9_macro_vars_transfer.sas (Part 1); 
options nodate nonumber ps=58;
proc sql noprint;
 select avg(weight) format=6.2 
    into :m_avg_wt 
    from sashelp.class;
 quit; 
%put Old way: &m_avg_wt ;
%put New way: &=m_avg_wt ;


                                                           The SAS System

60         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
60       ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
61         
62         *Ex9_macro_vars_transfer.sas (Part 1);
63         options nodate nonumber ps=58;
64         proc sql noprint;
65          select avg(weight) format=6.2
66             into :m_avg_wt
67             from sashelp.class;
68          quit;
NOTE: PROCEDURE SQL used (Total process time):
      real time           0.00 seconds
      cpu time            0.00 seconds
      

69         %put Old way: &m_avg_wt ;
Old way: 100.03
70         %put New way: &=m_avg_wt ;
New way: M_AVG_WT=100.03
71         
72         
73         ods html5 (id=saspy_internal) close;ods listing;
74         

                                                           The SAS System

75         


In [37]:
*Ex11_one_multiple_mvars_sql.sas (Part 1);
ods html close;
options nonotes nosource nodate nonumber;
proc sql noprint;
 select count(*)
        INTO :nobs
FROM SASHELP.CARS;
quit;
%put Number of Observations = %SYSFUNC(LEFT(&nobs));
run;

In [1]:
options nodate nonumber nosource;
ods html close;
options nonotes nosource nodate nonumber;
proc sql noprint;
 select distinct make
        INTO :make_list separated by ','
        FROM SASHELP.CARS;
quit;
%put &=make_list;
%put Nummber of car makes=&sqlobs;


5                                                          The SAS System                               14:42 Friday, March 22, 2024

24         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
24       ! ods graphics on / outputfmt=png;
NOTE: Writing HTML5(SASPY_INTERNAL) Body file: _TOMODS1
25         
26         options nodate nonumber nosource;
MAKE_LIST=Acura,Audi,BMW,Buick,Cadillac,Chevrolet,Chrysler,Dodge,Ford,GMC,Honda,Hummer,Hyundai,Infiniti,Isuzu,Jaguar,Jeep,Kia,Land 
Rover,Lexus,Lincoln,MINI,Mazda,Mercedes-Benz,Mercury,Mitsubishi,Nissan,Oldsmobile,Pontiac,Porsche,Saab,Saturn,Scion,Subaru,Suzuki,To
yota,Volkswagen,Volvo
Nummber of car makes=38

                                                           The SAS System

E3969440A681A2408885998500000003


In [2]:
proc freq data=sashelp.cars; tables make; run;

Make,Frequency,Percent,CumulativeFrequency,CumulativePercent
Acura,7,1.64,7,1.64
Audi,19,4.44,26,6.07
BMW,20,4.67,46,10.75
Buick,9,2.10,55,12.85
Cadillac,8,1.87,63,14.72
Chevrolet,27,6.31,90,21.03
Chrysler,15,3.50,105,24.53
Dodge,13,3.04,118,27.57
Ford,23,5.37,141,32.94
GMC,8,1.87,149,34.81


In [16]:
proc sql noprint ;
select distinct quote(trim(make)) into :make_q_list separated by ','
from sashelp.cars;
quit ;
%put &=make_q_list;

In [25]:

proc sql noprint ;
select distinct "'"||left(trim(make))||"'" into :make_sq_list
separated by ','
from sashelp.cars;
quit ;
%put &=make_sq_list;       

In [1]:
proc sql ;                                                                                                                              
      select quote(strip(Name))                                                                                                         
      INTO  :Starts_withC                                                                                              
    from sashelp.demographics                                                                                                           
      where Name LIKE 'C%';                                                                                                             
quit;                                                                                                                                   
%PUT &Starts_withC; 

""
"""CANADA"""
"""COSTA RICA"""
"""CUBA"""
"""CHILE"""
"""COLOMBIA"""
"""CZECH REPUBLIC"""
"""CROATIA"""
"""CAMEROON"""
"""CAPE VERDE"""
"""CENTRAL AFRICAN REP."""


##### Note for the code in the next cell
##### The macro variable follows an INTO clause and is preceded by a colon. Here the PROC SQL creates a series of macro variables, each with its own distinct value (often referred to as vertical list of macro variables of the form &MAKES1, and so on). 

##### When PROC SQL is executed, a series of macro variables including &SQLOBS is generated and placed in the most local symbol table.  The value of the macro variable &SQLOBS is the number of rows processed by the SELECT statement.

##### When you submit this macro, it is compiled and then stored in the default catalog work.sasmacr.


In [2]:
*Ex11_one_multiple_mvars_sql.sas (Part 3);
%let Put_title = List of Values into a Series of Macro Variables;
proc sql noprint;
 select distinct make
        INTO :makes1-
  FROM SASHELP.CARS ;
 %put Number of Rows: &sqlobs;
quit;
%macro reveal;
 %put &Put_title;
 %Do i=1 %TO &Sqlobs;
    %put &&makes&i;
  %end;
%mend reveal;
%reveal

[A Hands-on Introduction to SAS® Metadata DICTIONARY Tables
and SASHELP Views](https://www.mwsug.org/proceedings/2018/HW/MWSUG-2018-HW-9.pdf)

In [1]:
* Ex29_symputx_sql_into.sas (Part 1);
options nocenter nodate nonumber;
* Adapted from Simon (2017);
proc means data=sashelp.class noprint;
 var weight;
 output out=mystats mean=avweight;
 run;

data _null_;
 set mystats;
 call symputx('meanweight',avweight);
 run;

proc print data=sashelp.class noobs;
var name sex weight;
 where weight > &meanweight;
title 'Students weighing more than average weight';
title2 "Average weight:(%sysfunc(round(&meanweight, 0.01)) lbs)";
title3 '3 DATA/PROC Steps';
run;
title;

SAS Connection established. Subprocess id is 6344



Name,Sex,Weight
Alfred,M,112.5
Carol,F,102.5
Henry,M,102.5
Janet,F,112.5
Mary,F,112.0
Philip,M,150.0
Robert,M,128.0
Ronald,M,133.0
William,M,112.0


In [2]:
* Ex29_symputx_sql_into.sas (Part 2);
title 'Students weighing more than average weight';
title2 'PROC SQL - Just 1 Step';
proc sql ;
 select mean(weight) into: meanweight_x 
  FROM sashelp.class ;
 select name, sex, weight format=6.1
  FROM sashelp.class 
 having weight > &meanweight_x;
quit;